<a href="https://colab.research.google.com/github/ChabiMax/author_network/blob/main/Notebook%20scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tqdm import *
import time
import requests
import bs4
from collections import Counter
import json
import os
from google.colab import files
import pickle as pickle
import pandas as pd
import networkx as nx

In [3]:
# Extraction de la liste d'auteurs

URL = 'https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_chronologique'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary = {}
date_naissance = None
date_mort = None
name = 'init'
author_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    if not any(char.isdigit() for char in string) :
      if serie == 2 :
        serie = 0
        date_mort = 2025
        dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}
      serie = serie+1
      id = id+1
      name = string
      date_naissance = None
      date_mort = None
      if 'https://' in link.get('href'):
        author_link = link.get('href')
      else:
        author_link = 'https://fr.wikipedia.org' + link.get('href')
    else :
      if serie == 1 :
        serie = serie+1
        date_naissance = string
      else :
        if serie == 2 :
          serie = 0
          date_mort = string
          dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}

# Nettoyage manuel
unwanted_list = list(range(0,5))
unwanted_list += list(range(1793,1806))
unwanted_list += [key for key in dictionary.keys() if 'modifier' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'siècle' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'ms.' in dictionary[key]['name']]
dictionary = dict((key, dictionary[key]) for key in dictionary.keys() if key not in unwanted_list)



In [4]:
df = pd.DataFrame.from_dict(dictionary).T

In [5]:
df

,name,link,date_mort,date_naissance
6,Turold,https://fr.wikipedia.org/wiki/Turold,2025,None
10,Philippe de Thaon,https://fr.wikipedia.org/wiki/Philippe_de_Thaon,2025,None
14,Benoît de Sainte-Maure,https://fr.wikipedia.org/wiki/Beno%C3%AEt_de_S...,2025,None
16,Jean Bodel,https://fr.wikipedia.org/wiki/Jean_Bodel,1165,None
17,Wace,https://fr.wikipedia.org/wiki/Wace,1174,1100
...,...,...,...,...
1785,Édouard Louis,https://fr.wikipedia.org/wiki/%C3%89douard_Louis,2025,1992
1786,Myriam Thibault,https://fr.wikipedia.org/wiki/Myriam_Thibault,2025,1993
1787,Maud Ventura,https://fr.wikipedia.org/wiki/Maud_Ventura,2025,1993
1788,Carmen Bramly,https://fr.wikipedia.org/wiki/Carmen_Bramly,2025,1995


NB: La date de naissance de Jean Bodel est renseignée sur Wikipedia mais n'apparaît pas dans la base scraped. A corriger.

In [6]:
 # Extraction de la liste des genres littéraires

URL = 'https://fr.wikipedia.org/wiki/Genre_litt%C3%A9raire'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary_genres = {}
name = 'init'
genre_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    id = id+1
    name = string
    if 'https://' in link.get('href'):
      genre_link = link.get('href')
    else:
      genre_link = 'https://fr.wikipedia.org' + link.get('href')
    dictionary_genres[id] = {'name' : name, 'link' : genre_link}

# Nettoyage manuel
unwanted_list = list(range(0,49))
unwanted_list += list(range(278,344))
unwanted_list += [key for key in dictionary_genres.keys() if 'modifier' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if dictionary_genres[key]['link'] in [dictionary[key_bis]['link'] for key_bis in dictionary.keys()]]
unwanted_list += [key for key in dictionary_genres.keys() if any(char.isdigit() for char in dictionary_genres[key]['name'])]
unwanted_list += list(range(285,len(dictionary_genres.keys())))
dictionary_genres = dict((key, dictionary_genres[key]) for key in dictionary_genres.keys() if key not in unwanted_list)


In [7]:
# Parcours des pages et création du réseau

author_links = [dictionary[key]['link'] for key in dictionary.keys()]
genre_links = [dictionary_genres[key]['link'] for key in dictionary_genres.keys()]

for id, values in tqdm_notebook(dictionary.items()) :
  link = values['link']
  response = requests.get(link)
  soup = bs4.BeautifulSoup(response.text, 'html.parser')
  links = soup.find(id="bodyContent").select('a')
  network_aslink = []
  net_genre_aslink = []
  for link in links:
    if link.get('href') != None:
      if 'https://' in link.get('href'):
        link = link.get('href')
      else:
        link = 'https://fr.wikipedia.org' + link.get('href')
    if link in author_links :
      network_aslink.append(link)
    if link in genre_links :
      net_genre_aslink.append(link)
  network = []
  net_genre = []
  for link in network_aslink :
    for id_bis, values_bis in dictionary.items() :
      if link == values_bis['link'] :
        network.append(id_bis)
  for link in net_genre_aslink :
    for id_bis, values_bis in dictionary_genres.items() :
      if link == values_bis['link'] :
        net_genre.append(id_bis)
  counted_network = Counter(network)
  counted_net_genre = Counter(net_genre)
  dictionary[id]['taille_article'] = len(str(soup))
  dictionary[id]['network'] = list(counted_network.keys())
  dictionary[id]['weights'] = list(counted_network.values())
  dictionary[id]['net_genre'] = [list(counted_net_genre.keys()) if len(list(counted_net_genre.keys())) != 0 else None]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1581 [00:00<?, ?it/s]

In [8]:
# On fait un réseau centré autour du genre

for id, values in tqdm_notebook(dictionary_genres.items()) :
  try :
    dictionary_genres[id]['Auteur_affilié'] = [dictionary[key] for key in dictionary.keys() if id in dictionary[key][net_genre]]
  except :
    pass

# On complète dictionaire

for id, values in dictionary.items() :
  dictionary[id]['meme_genre'] = []
  dictionary[id]['contemporain'] = []
vu = []
for id, values in tqdm_notebook(dictionary.items()) :
  for id_bis, values_bis in dictionary.items() :
    if id_bis != vu :
      if id == id_bis :
        pass
      else :
        if any(i in dictionary[id]['net_genre'] for i in dictionary[id_bis]['net_genre']) :
          dictionary[id]['meme_genre'].append(id_bis)
          dictionary[id_bis]['meme_genre'].append(id)
        mort_1 = [dictionary[id]['date_mort'] if dictionary[id]['date_mort'] != None else 9999]
        mort_2 = [dictionary[id_bis]['date_mort'] if dictionary[id_bis]['date_mort'] != None else 9999]
        naissance_1 = [dictionary[id]['date_naissance'] if dictionary[id]['date_mort'] != None else 0]
        naissance_2 = [dictionary[id_bis]['date_naissance'] if dictionary[id_bis]['date_mort'] != None else 0]
        try:
          if (mort_1 > naissance_2 and naissance_1 < mort_2) or (mort_2 > naissance_1 and naissance_2 < mort_1):
            dictionary[id]['contemporain'].append(id_bis)
            dictionary[id_bis]['contemporain'].append(id)
        except :
          pass
    vu.append(id)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/208 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/1581 [00:00<?, ?it/s]

In [9]:
for id in dictionary.keys() :
  if dictionary[id]['net_genre'][0] == None :
    dictionary[id]['meme_genre'] = []
  dictionary[id]["meme_genre"] = list(set(dictionary[id]["meme_genre"]))
  dictionary[id]["contemporain"] = list(set(dictionary[id]["contemporain"]))

In [10]:
df_net= pd.DataFrame.from_dict(dictionary).T
df_net.head()

,name,link,date_mort,date_naissance,taille_article,network,weights,net_genre,meme_genre,contemporain
6,Turold,https://fr.wikipedia.org/wiki/Turold,2025,None,72697,[],[],[None],[],[]
10,Philippe de Thaon,https://fr.wikipedia.org/wiki/Philippe_de_Thaon,2025,None,67928,[],[],[None],[],[]
14,Benoît de Sainte-Maure,https://fr.wikipedia.org/wiki/Beno%C3%AEt_de_S...,2025,None,82409,[17],[1],[[94]],"[1538, 1542, 1545, 1550, 529, 1557, 26, 1055, ...",[]
16,Jean Bodel,https://fr.wikipedia.org/wiki/Jean_Bodel,1165,None,72375,"[23, 43]","[1, 1]","[[129, 164, 69]]",[],[]
17,Wace,https://fr.wikipedia.org/wiki/Wace,1174,1100,98815,[19],[2],"[[56, 128]]","[536, 554]","[18, 19, 20, 22, 23, 24]"


In [12]:
node_table = df_net
node_table['Id'] = node_table.index
node_table = node_table.rename(columns={'name': 'Label'})

In [13]:
node_table.head()

,Label,link,date_mort,date_naissance,taille_article,network,weights,net_genre,meme_genre,contemporain,Id
6,Turold,https://fr.wikipedia.org/wiki/Turold,2025,None,72697,[],[],[None],[],[],6
10,Philippe de Thaon,https://fr.wikipedia.org/wiki/Philippe_de_Thaon,2025,None,67928,[],[],[None],[],[],10
14,Benoît de Sainte-Maure,https://fr.wikipedia.org/wiki/Beno%C3%AEt_de_S...,2025,None,82409,[17],[1],[[94]],"[1538, 1542, 1545, 1550, 529, 1557, 26, 1055, ...",[],14
16,Jean Bodel,https://fr.wikipedia.org/wiki/Jean_Bodel,1165,None,72375,"[23, 43]","[1, 1]","[[129, 164, 69]]",[],[],16
17,Wace,https://fr.wikipedia.org/wiki/Wace,1174,1100,98815,[19],[2],"[[56, 128]]","[536, 554]","[18, 19, 20, 22, 23, 24]",17


In [21]:
pd.DataFrame.to_csv(node_table, 'node_table.csv', index=False)
files.download('node_table.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
id = 0
link_table = dict()
for source in dictionary.keys() :
  id_repere = id
  for target in dictionary[source]['network'] :
    id += 1
    link_table[id] = {'source' : source, 'target' : target}
  for weight in dictionary[source]['weights'] :
    id_repere += 1
    link_table[id_repere]['weight'] = weight


In [16]:
edge_table = pd.DataFrame(link_table).T

In [17]:
pd.DataFrame.to_csv(edge_table, 'edge_table.csv', index=False)
files.download('edge_table.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# On sauvegarde sur son ordi en pickle (ne pas oublier d'upload ensuite sur github)

with open('dictionary_pickle.pkl', 'wb') as f:
    pickle.dump(dictionary, f)
files.download('dictionary_pickle.pkl')